
# Notebook 2 — The Discount Factor & Tipping Points

**Course notebook** using `DICE.py`.  
We work on two blocks from the slides: **“The discount factor”** and **“Tipping points.”**

## Summary of the notebook
- **0)** Load `DICE.py` and run a baseline.
- **1)** **The discount factor controversy**  
  1-A) From discount factor to interest rate (and quick coding exercise).  
  1-B) Real-world interest rates; pick 3 values for `p.rho` (including Stern’s low value).  
  1-C) Compute the optimal policy for the 3 scenarios.  
  1-D) Plot and compare **abatement**, **carbon tax**, **damages**, **temperatures**.  
  1-E) Short comments.
- **2)** **The role of tipping points**  
  2-A) Using `p.user_damage_fn` to define custom damages (baseline Nordhaus; double coefficient).  
  2-B) Plot outcomes.  
  2-C) Interpret.  
  2-D) Implement a **Weitzman** damage function and solve.  
  2-E) Compare figures.  
  2-F) Comment.  
  2-G) Add a **kink** at 3°C (damages double above the threshold) and interpret.



## 0) Load `DICE.py`
Make sure `DICE.py` sits next to this notebook (or adjust the import path accordingly).


In [ ]:
%load_ext autoreload
%autoreload 2
    
from DICE import Params, init_states, update_path, run_optimal_policy, mat_to_df
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Baseline initialization
p       = Params()
sim     = init_states(p)
timevec = range(1, p.nT)
sim     = update_path(sim, timevec, p)

# Peek at the dataframe if you want
df = mat_to_df(sim, p)
df.head()


## 1) The Discount Factor Controversy

**Context.**  
The discount factor determines how we value the future. A higher pure rate of time preference $\rho$ means we discount the future more heavily—typically implying **lower** near-term abatement. A lower $\rho$ (as advocated by **Stern**) implies **strong** and **early** mitigation.

- **Nordhaus** (e.g., DICE): often uses $\rho \approx 1.5\%$ per year (plus growth & curvature terms in the consumption Euler equation).  
- **Stern (2006, Stern Review)**: argues for a **near-zero** pure rate of time preference ($\rho \approx 0.1\%$ per year).  
  Link to the **Stern Review**: https://webarchive.nationalarchives.gov.uk/ukgwa/20100407172811/http://www.hm-treasury.gov.uk/stern_review_report.htm

This debate is ethical as much as technical: how much should current generations **value future generations' welfare**?



### 1-A) From discount factor to interest rate

Let $\beta = \dfrac{1}{1+\rho}$ denote the planner’s discount factor. Under standard assumptions, the (approximate) real interest rate is
$$
r \approx \rho + \gamma g,
$$
where $g$ is per-capita consumption growth and $\gamma$ is the CRRA parameter (inverse of IES).

**Exercise.**
1. Read the current discounting parameters from `p`: `p.rho` and `p.gamma`.
2. Assume a plausible growth rate of consumption, e.g. $g=2\%$.
3. Compute and print $\beta$ and $r$. (Feel free to experiment with other values of $g$.)


In [ ]:

# >>> Your code here <<<
rho = p.rho       # pure rate of time preference
gamma = p.gamma   # CRRA coefficient (1/IES)
g = 0.02          # consumption growth (try 0.01, 0.015, 0.02...)
# ...


### 1-B) Real-world interest rates & choosing three $\rho$ values

Long-run **real** interest rates are typically in the ballpark of 1–4% annually (varying by country and period). One can have a glance at:  
- US data on long term government bonds https://fred.stlouisfed.org/series/IRLTLT01USA156N
- Or a global R https://cepr.org/voxeu/columns/global-r

For this exercise, consider three values for the pure rate of time preference:
- **Nordhaus-like**: $\rho = 0.015$ (1.5%)
- **Intermediate**: $\rho = 0.005$ (0.5%)
- **Stern-like**: $\rho = 0.001$ (0.1%) — often summarized as $\beta \approx 0.999$.

We will solve the optimal policy for each. Define here simply the corresponding $\rho$.

First, change the calibration of rho for the other sceanrios.


In [ ]:
# >>> Your code here <<<
pNordhaus     = Params()
pIntermediate = Params()
pStern        = Params()
# a twist to get calculation manageable, otherwise the planner windows expands a lot
pIntermediate.toly   = 0.01
pStern.toly          = 0.01
# ...


### 1-C) Compute the optimal policy for the three scenarios

We optimize over **one control** to reduce the computation time: the abatement rate $\mu_t$.


In [ ]:
# >>> Your code here <<<
bounds_s   = [(0, 1)]
control_id = [pNordhaus.i_mu]
path_opt_Nordhaus = run_optimal_policy(sim.copy(), timevec, pNordhaus, bounds_s, control_id)
# ...


### 1-D) Plot: abatement, carbon tax, damages, temperatures

**Exercise.** Create comparison plots across the three discounting scenarios.


In [ ]:
# >>> Your code here <<<

plt.subplot(1, 4, 1)
# plot abatement
# var name: "mu"

plt.subplot(1, 4, 2)
# plot damages
# damages = p.a2 * (sim[0:,p.i_T_AT] ** p.a3)

plt.subplot(1, 4, 3)
# plot carbon tax
# var name: "Tax"

plt.subplot(1, 4, 4)
# plot temperatures
# var name: "T_AT"

plt.show()



### 1-E) Comment (short paragraph)

**Exercise.** Write a short paragraph:
- How does lowering $\rho$ alter abatement paths and taxes?  
- What happens to temperatures and damages?  
- Do your results align with the intuition from the slides?


> ✍️ You written answer here.


## 2) The Role of Tipping Points

**Slides recap.**  
Tipping points refer to abrupt, potentially irreversible shifts in the climate system (e.g., ice-sheet collapse, AMOC changes, permafrost thaw). They imply **nonlinear** and **fat-tailed** risks. Smooth quadratic damages may understate extreme outcomes.



### 2-A) Modifying the damage function

We start with a **Nordhaus-like** baseline damage function:  
$$
D(T) = a_2 \, T^{a_3}, \quad 
\text{with } a_2 = 0.00236,\; a_3 = 2.
$$

In the extended code, an **additional threshold-activated term** can also apply:  
$$
D(T) =
a_2 \, T^{a_3} \;+\;
\mathbf{1}_{\{T > a_6\}} \; a_4 \, T^{a_5},
$$
where $\mathbf{1}_{\{T > a_6\}}$ is an indicator that activates once temperature exceeds the threshold $a_6$.  

**Exercise.**
1. Implement and assign a function for the baseline Nordhaus form $D(T) = a_2 T^{a_3}$.  
2. Solve for the optimal policy under this baseline.  
3. Modify the function to **double the baseline coefficient** (i.e., $2 \times a_2$) and solve again.  



In [ ]:
# This code is incomplete, but shows you how to partiallly do 1.
# Baseline Nordhaus damages via user function
pN = Params()

#Simulate
timevec = range(1, p.nT)
simN    = init_states(pN)

bounds_s   = (0,1)
control_id = [pN.i_mu]
#path_opt_s = run_optimal_policy(simN.copy(), timevec, pN, bounds_s, control_id)

#...



### 2-B) Plot outcomes (damages, tax, abatement)

**Exercise.** Compare **baseline** vs **doubled** damages on damages, tax, and abatement (assuming implementation of optimal abatement).


In [ ]:
# Your code here.
plt.subplot(1, 4, 1)
# plot abatement
# var name: "mu"
#plt.plot(path_opt1[:,p.i_time], path_opt[:, p.i_mu], label="Damage x1")
#plt.plot(path_opt_Intermediate[:,p.i_time],path_opt[:, p.i_mu], label="Damage x2")
plt.title("Abatement")
plt.legend()

plt.subplot(1, 4, 2)
# plot damages
#damages1 = 100*p.a2 * (path[0:,p.i_T_AT] ** p.a3)

plt.title("Damages")

plt.subplot(1, 4, 3)
# plot carbon tax
# var name: "Tax"

plt.title("Tax")

plt.subplot(1, 4, 4)
# plot temperatures
# var name: "T_AT"

plt.title("T_AT")

plt.show()


### 2-C) Interpret (short paragraph)

**Exercise.** Explain how higher damages affect optimal policy:
- How do tax and abatement shift over time?
- How does the temperature trajectory react?


> ✍️ You written answer here.


### 2-D) Weitzman-style damages (fat tails)

A stylized **Weitzman**-type specification introduces **fat tails** (faster-than-quadratic growth at high $T$).  
For illustration, consider:
$$
D_{W}(T) \;=\; a_2 T^{a_3} \;+\; b \, T^{c}
$$
with a small $b=5.0703e-06$ capturing tail risks with polynomial degree $c=6.754$.

A common *qualitative* comparison:
|                      | <span style="color:red;">Ω(1)</span> | <span style="color:red;">Ω(3)</span> | <span style="color:red;">Ω(5)</span> |
|----------------------|-------------------------------------:|-------------------------------------:|-------------------------------------:|
| *b* = 0 — Nordhaus (2017) | 0.9976 | 0.9792 | 0.9443 |
| *b* > 0 — Weitzman (2012)        | 0.9976 | 0.9712 | 0.7544 |


**Exercise.**
1. Report in a graph the Damages for each damage function considering $T^{AT} \in [0;4]$.
2. Implement a Weitzman-like affecting new values for $a_4$ and $a_5$.  
3. Solve the optimal policy.


In [ ]:
pW = Params()
# >>> Your code here <<<

def weitzmann_damage(T, p: Params):
    T = np.asarray(T)
    return p.a2 * (T**p.a3) # update here

# continue code


### 2-E) Compare figures (Nordhaus vs. Weitzman)

**Exercise.** Plot and compare damages, carbon tax, and abatement for `pN` (Nordhaus) vs `pW` (Weitzman).


In [ ]:
# >>> Your code here <<<


### 2-F) Comment (short paragraph)

**Exercise.** Discuss the policy differences implied by fatter tails:
- Are taxes/abatement higher and earlier?
- How sensitive are results to \(\kappa\)? Try a few values.


> ✍️ You written answer here.


### 2-G) Introduce a **kink** at 3°C

**Idea (slides):** if $T \le 3^{\circ}C$ use the baseline quadratic damages; if $T>3^{\circ}C$ **double** damages (toy tipping point).

$$
D_{\text{kink}}(T) \;=\; 
\begin{cases}
a_2 T^{a_3}, & T \le 3 \\[0.2em]
2\,a_2 T^{a_3}, & T > 3
\end{cases}
$$

This formulation is inspired by Fillon, Guivarch and Taconet (2023) https://www.sciencedirect.com/science/article/abs/pii/S0095069623000682

**Exercise.**
1. Implement `kink_damage` as above.  
2. Solve the optimal policy.  
3. Plot vs. `pN` and interpret.


In [ ]:
# >>> Your code here <<<

> ✍️ You written answer here.